In [ ]:
import os

allDirs=[x[0] for x in os.walk('/Users/iuliailie/UCD/mtsdata (2)/')]
len(allDirs)
allDirs
for lagsPermitted in [10, 15, 20]:#file save is repeated for all the number of lags desired
   # allDirs=['NetFlow','ECG','Wafer','CMUsubject16','KickvsPunch','WalkvsRun'] 
     #problem names can be obtained from directory
    for i in range(0,len(allDirs)):
        dirName=allDirs[i].split('/')[-1]
        pandaMTSC_train=pd.read_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+'train.txt',sep='\t',header=None)
        pandaMTSC_test=pd.read_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+'test.txt',sep='\t',header=None)
        print(dirName)
        #obtain class flags
        uniqClasses=pandaMTSC_train[pandaMTSC_train.columns[-2]].unique()
       #train here is given by all features in train file
        XVert=pandaMTSC_train[pandaMTSC_train.columns[0:-2]]
        #Y here is the class flag
        YVert=pandaMTSC_train[pandaMTSC_train.columns[-2]]

        # Class trees for target feature determination
        
        import numpy as np
        import matplotlib.pyplot as plt

        from sklearn.datasets import make_classification
        from sklearn.ensemble import ExtraTreesClassifier

        # Build a classification task using n informative features


        # Build a forest and compute the feature importances
        forest = ExtraTreesClassifier(n_estimators=250,
                                      random_state=0)
        X=XVert
        y=YVert
        forest.fit(X, y)
        
        #obtained forest fit and important features
        importances = forest.feature_importances_
        std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                     axis=0)
        #sort by importance
        indices = np.argsort(importances)[::-1]
        
        #most important is assigned as target
        target_index=indices[0]
        #all others become candidate regression features
        dependent_indices=[]
        for i in range(1,len(indices)):
            dependent_indices.append(indices[i])
        dependent_indices.sort()


        #obtaining all time series in train file
        ts_indices=pandaMTSC_train[pandaMTSC_train.columns[-1]].unique()

        pandaMTSCLags_train=pd.DataFrame([])
        pandaMTSC_train.shape[1]

        
        NameAdded=str(lagsPermitted)+"Lags"
        pandaMTSCLags_test=pd.DataFrame([])

        #obtaining all time series groups in train file
        gr=pandaMTSC_train.groupby(pandaMTSC_train[pandaMTSC_train.columns[-1]])

        for j in range(0,len(ts_indices)):

            pandaOne_TS_train=gr.get_group(ts_indices[j])
            pandaOne_TS_trainLags=pd.DataFrame([])
            
            #generating lags
            for i in range(0,int(lagsPermitted)+1):
                    pandaOne_TS_trainLags=pd.concat([pandaOne_TS_trainLags,
                                                         pandaOne_TS_train[pandaOne_TS_train.columns[0:-2]].shift(i)],axis=1)
            pandaOne_TS_trainLags=pd.concat([pandaOne_TS_trainLags,
                                             pandaOne_TS_train[pandaOne_TS_train.columns[-2:]]],axis=1)

            pandaMTSCLags_train=pd.concat([pandaMTSCLags_train,pandaOne_TS_trainLags],axis=0)

        pandaMTSCLags_train=pandaMTSCLags_train.dropna()
        pandaMTSCLags_train.columns=np.arange(pandaMTSCLags_train.shape[1])

        pandaMTSCLags_train.to_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+'_AllClassesTrain'+NameAdded+'.txt',
                          header=None,
                          sep='\t',
                          index=None)

        ts_indicesTest=pandaMTSC_test[pandaMTSC_test.columns[-1]].unique()


        pandaMTSC_test.shape[1]
       # lagsPermitted=24/pandaMTSC_test[pandaMTSC_test.columns[0:-2]].shape[1]

        gr=pandaMTSC_test.groupby(pandaMTSC_test[pandaMTSC_test.columns[-1]])
        for j in range(0,len(ts_indicesTest)):

            pandaOne_TS_test=gr.get_group(ts_indicesTest[j])
            pandaOne_TS_testLags=pd.DataFrame([])
            for i in range(0,int(lagsPermitted)+1):
                    pandaOne_TS_testLags=pd.concat([pandaOne_TS_testLags,
                                                             pandaOne_TS_test[pandaOne_TS_test.columns[0:-2]].shift(i)],axis=1)
            pandaOne_TS_testLags=pd.concat([pandaOne_TS_testLags,
                                                             pandaOne_TS_test[pandaOne_TS_test.columns[-2:]]],axis=1)
            pandaMTSCLags_test=pd.concat([pandaMTSCLags_test,pandaOne_TS_testLags],axis=0)

        pandaMTSCLags_test=pandaMTSCLags_test.dropna()
        pandaMTSCLags_test.columns=np.arange(pandaMTSCLags_test.shape[1])



        pandaMTSCLags_test.to_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+'_AllClassesTest'+NameAdded+'.txt',
                          header=None,
                          sep='\t',
                          index=None)



        classes=pandaMTSCLags_train[pandaMTSCLags_train.columns[-2]].unique()
        Cl_group=pandaMTSCLags_train.groupby(pandaMTSCLags_train.columns[-2])
       
        #saving train test files by class flag

        for i in range(0,len(classes)):
            gr_cl=Cl_group.get_group(classes[i])
            #shuffling samples over all time series and saving to X->Y format for CMAGEP regression  
            gr_cl=gr_cl.sample(len(gr_cl))
            Ytrain=pd.DataFrame(gr_cl[gr_cl.columns[target_index]])
            Ytrain.to_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+str(int(classes[i]))+'_YTrain'+NameAdded+'.txt',
                          header=None,
                          sep='\t',
                          index=None)
            XTrain=gr_cl.drop([gr_cl.columns[target_index],gr_cl.columns[-1],gr_cl.columns[-2]],axis=1);
            XTrain.to_csv('/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/'+dirName+'/'+dirName+str(int(classes[i]))+'_XTrain'+NameAdded+'.txt',
                          header=None,
                          sep='\t',
                          index=None)

    #saving problem specification and data transformations
        f= open("/Users/iulia/UCD laptop backup/UCD/mtsdata (2)/"+dirName+"/problemSettings"+NameAdded+".txt","a+")
        f.write("target feature %d\r\n" % target_index)
        f.write("original number of dimensions %d\r\n" % len(importances))
        f.write("final number of dimensions %d\r\n" % int(pandaMTSCLags_train.shape[1]-2))
        f.close()
